Goal: this notebook will produce the time series of power density with a breakdown of the contribution from each decay type from all known decay chains. 

In [1]:
import astropy.units as units
import astropy.constants as constants
import matplotlib.pyplot as plt
import sympy as sym
from sympy.abc import *
import numpy as np 
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import requests
import re
import os
import subprocess
import urllib.request
from bs4 import BeautifulSoup
import csv
cwd = subprocess.os.getcwd()
#%matplotlib notebook #incompatible with mpmath

Error importing optional module geopandas
Traceback (most recent call last):
  File "c:\Users\engin\anaconda3\lib\site-packages\_plotly_utils\optional_imports.py", line 30, in get_module
    return import_module(name)
  File "c:\Users\engin\anaconda3\lib\importlib\__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1014, in _gcd_import
  File "<frozen importlib._bootstrap>", line 991, in _find_and_load
  File "<frozen importlib._bootstrap>", line 975, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 671, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 783, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "c:\Users\engin\anaconda3\lib\site-packages\geopandas\__init__.py", line 3, in <module>
    from geopandas.geoseries import GeoSeries  # noqa
  File "c:\Users\engin\anaconda3\lib\site-package

#Import half-lifes and energy per emission from databases
[Zotero Collection](https://www.zotero.org/groups/4549380/batteries/collections/59RQX9TX) / [Atomic Mass Data Center (AMDC)](https://www-nds.iaea.org/amdc/)



##Nubase2020

In [2]:
url = "https://www-nds.iaea.org/amdc/ame2020/nubase_3.mas20.txt"
response = requests.get(url)
nubase = np.array(response.text.split('\n'))
nubase = nubase[25:]
column_dict = {'AAA': np.arange(1,4), 'ZZZi':np.arange(5,9),
'A El': np.arange(11,17), 's': np.array([17]), 
'Mass #': np.arange(19,32), 'dMass #': np.arange(32,43),
'Exc #': np.arange(43,55), 'dE #': np.arange(55,66),
'Orig': np.arange(66,68), 'Isom.Unc': np.array([68]),
'Isom.Inv': np.array([69]), 'T #': np.arange(70, 79),
'unit T': np.arange(78, 81), 'dT': np.arange(81, 89),
'Jpi */#/T=': np.arange(88, 103), 
'Ensdf year': np.arange(102, 105), 
'Discovery': np.arange(114, 119), 'BR': np.arange(119, 210)}
#make an array of lists of the columns

def clean(array):
    #turn the array of characters into a string and remove spaces
    array = ''.join(array).replace(' ', '')
    try:
        array = array.astype(float)
    except:
        pass
    return array

#Turn the list of strings into a multidimensional array

#get substring from each item in array
def get_substring(array, start, end):
    #start and end are the indices of the substring
    #returns a list of the substring
    try:
        return [item[start:end] for item in array]
    except:
        print(len(item), end)
        return ("error")

def make_dict_from_string_array(column_dict, string_array):
    #column_dict is a dictionary of the column names and the indices
    #string_array is a list of strings
    #returns a dictionary of the column names and the values
    nubase_dict = {}
    for key, columns in column_dict.items():
        nubase_dict[key] = get_substring(string_array, columns[0], columns[-1])
    return nubase_dict

nubase_df = pd.DataFrame(make_dict_from_string_array(column_dict, nubase))

units_dict = {'s': units.s, 'h': units.h, 'd': units.d, 'm' : units.minute, 
              'y': units.year, 'ky': units.kiloyear, 'My': units.megayear,
              'as': units.attosecond, 'ys': units.yoctosecond, 'zs': units.zeptosecond,
              'ms': units.ms, 'ns': units.ns, 'us': units.us, 'μs': units.microsecond,
              'ps': units.ps, 'fs': units.fs, 'My': units.myr, 'Gy': units.gigayear,
              'ty': units.Tyr, 'py': units.Pyr, 'ny': units.nanoyear, 'Yy': units.yottayear,
              'Zy': units.zettayear, 'Ey': units.Eyr,}

def convert_half_life(number, unit, units_dict):
  try: 
    half_life = float(number) * units_dict[unit].to(units.year)
  except: #isotope is stable
    half_life = 'unknown'
  return half_life

def remove_symbols(string, symbol_list):
  for symbol in symbol_list:
    string = string.replace(symbol, "")
  return string

sym_list = [" ", "#", "*"]

half_lives_seconds = [convert_half_life(remove_symbols(t, sym_list), 
                    remove_symbols(nubase_df['unit T'][row], sym_list), 
                    units_dict) for row, t in enumerate(nubase_df['T #'])]
nubase_df['T in s'] = half_lives_seconds
nubase_df['A El'] = [symbol.replace(" ","") for symbol in nubase_df['A El']]
nubase_df.to_csv('nubase_df.csv')
nubase_df

,AAA,ZZZi,A El,s,Mass #,dMass #,Exc #,dE #,Orig,Isom.Unc,Isom.Inv,T #,unit T,dT,Jpi */#/T=,Ensdf year,Discovery,BR,T in s
0,01,000,1n,,8071.3181,0.0004,,,,,,609.8,s,0.6,1/2+*,06,1932,B-=100,1.93234e-05
1,01,010,1H,,7288.971064,0.000013,,,,,,stbl,,,1/2+*,06,1920,IS=99.9855 78,unknown
2,02,010,2H,,13135.722895,0.000015,,,,,,stbl,,,1+*,03,1932,IS=0.0145 78,unknown
3,03,010,3H,,14949.81090,0.00008,,,,,,12.32,y,0.02,1/2+*,00,1934,B-=100,12.32
4,03,020,3He,,14931.21888,0.00006,,,,,,stbl,,,1/2+*,98,1934,IS=0.0002 2,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5839,93,180,293Og,,98800#,710#,,,N,,,1#,ms,,,00,2010,A ?,3.16881e-11
5840,94,170,294Ts,,96400#,590#,,,,,,70,ms,30,,19,2010,A=100,2.21817e-09
5841,94,180,294Og,,99320#,550#,,,,,,0.7,ms,0.3,0+,05,2004,A~100; SF ?,2.21817e-11
5842,95,180,295Og,,01370#,660#,,,,,,680,ms,540,,,2006,A~100,2.15479e-08


##AME2020

In [3]:
url = "https://www-nds.iaea.org/amdc/ame2020/mass_1.mas20.txt"
response = requests.get(url)
Atomic_mass_table_2020 = response.text
#Now we want to convert a string to a pandas dataframe
Atomic_mass_table_2020 = list(Atomic_mass_table_2020.split('\n'))
split_table = Atomic_mass_table_2020[36:]

def clean_uncertainty(uncertainty):
    uncertainty = uncertainty.replace('.', '')
    uncertainty = uncertainty.replace('a', '0')
    uncertainty = uncertainty.replace('#', '')
    uncertainty = float("0." + uncertainty)
    return uncertainty

def clean_row(row):
    while True:
        try:
            row[2] = int(row[2])
            number = row.pop(0)
        except:
            row.insert(0, number)
            break
    #The above while loop ensures the first column is the number of neutrons
    try: 
        row[4] = float(row[4]) #if this fails, we the row is valid
        row.insert(4, "NA")
    except:
        pass
    try:
        row[10] = row[10].replace('#', '')
        row[10] = float(row[10]) #This means element 9 is *
    except:
        row.insert(11, "NA")
    #if not (len(row) == 15):
    #    print(row, len(row), row[9])
    #print(len(row), row)
    row[12] = float(row[12]) + clean_uncertainty(row[13])
    #this number was formatted weirdly, so we need to clean it up
    row.pop(13)
    
    return row

for i in range(len(split_table)):
    try:
        split_table[i] = clean_row(split_table[i].split())
    except:
        print(split_table[i].split())
#We know the column names are on row 34 (0-indexed)
#now we will make a pandas dataframe from the list of rows
#Annoyingly, the column names don't include the uncertainties, so we need to add them
my_column_names = ["N", "Z", "A", "Elt.", "Orig.", "Mass excess (keV)", "Mass excess (uncertainty)",
 "Binding energy per nucleon (keV)", "Binding energy per nucleon (uncertainty)", 
 "Beta-decay Type", "Beta-decay energy (keV)", 
 "Beta-decay energy (uncertainty)", "Atomic mass (μu)", 
 "Atomic mass (uncertainty)"]
 #Now we want to write the dataframe to a csv file
AM_table = pd.DataFrame(split_table, columns = my_column_names)
'''
First we will convert the atomic mass numbers to integers then concatenate them
with their chemical symbol. 
'''
def concat_for_AM_table(a, b):
  try: 
    a = int(a)
  except:
    a = 0
  return (str(a) + str(b))

AM_table['A Elt.'] = [concat_for_AM_table(A, AM_table['Elt.'][row]).replace(" ","") for row, A 
                 in enumerate(AM_table['A'])]
AM_table.to_csv("Atomic_mass_table_2020.csv")
AM_table

[]


,N,Z,A,Elt.,Orig.,Mass excess (keV),Mass excess (uncertainty),Binding energy per nucleon (keV),Binding energy per nucleon (uncertainty),Beta-decay Type,Beta-decay energy (keV),Beta-decay energy (uncertainty),Atomic mass (μu),Atomic mass (uncertainty),A Elt.
0,1,0.0,1.0,n,NA,8071.32,0.00044,0.0,0.0,B-,782.347,0.0004,1.008665,0.00047,1n
1,0,1.0,1.0,H,NA,7288.97,0.000013,0.0,0.0,B-,*,NA,1.007825,0.000014,1H
2,1,1.0,2.0,H,NA,13135.7,0.000015,1112.2831,0.0002,B-,*,NA,2.014102,0.000015,2H
3,2,1.0,3.0,H,NA,14949.8,0.00008,2827.2654,0.0003,B-,18.592,0.00006,3.016049,0.00008,3H
4,1,2.0,3.0,He,NA,14931.2,0.00006,2572.68044,0.00015,B-,-13736,2000#,3.016029,0.00006,3He
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3554,175,118.0,293.0,Og,-a,198802#,709#,7078#,2#,B-,*,NA,293.213423,761#,293Og
3555,177,117.0,294.0,Ts,-a,196397#,593#,7092#,2#,B-,-2923,811#,294.210840,637#,294Ts
3556,176,118.0,294.0,Og,-a,199320#,553#,7079#,2#,B-,*,NA,294.213979,594#,294Og
3557,177,118.0,295.0,Og,-a,201369#,655#,7076#,2#,B-,*,NA,295.216178,703#,295Og


##Half-life vs. Beta Emission

##Import average Decay Energies

In [4]:
#There is a new file we can convert to a pandas dataframe that has the average beta decay
#source: https://www.doseinfo-radar.com/RADARDecay.html
#source on Google Drive: https://docs.google.com/spreadsheets/d/1D_mDJWseMenElq68H10CD0cncqnBdhZ6EffqfNk0_XU/edit?usp=sharing
file_path = os.getcwd() + "\ImportedData\Radardec4OL.xls"
RADAR_df = pd.read_excel(file_path)
RADAR_df = RADAR_df.iloc[3:,1:]
RADAR_df.columns = list(("A", "ELEM", "Z", "Radiation Decay Mode", 
                    "Half-Life", "Half-Life Units", "Rad. Type", 
                    "Energy (keV)", "Radiation Intensity (%)"))
RADAR_A = list(RADAR_df['A'])
RADAR_ELEM = list(RADAR_df['ELEM'])

def combine_RADAR(RADAR_A, RADAR_ELEM):
    if len(RADAR_ELEM) > 1:
        RADAR_ELEM = list(RADAR_ELEM)
        RADAR_ELEM[1] = str.lower(RADAR_ELEM[1])
        RADAR_ELEM = ''.join(RADAR_ELEM)
    return str(RADAR_A) + str(RADAR_ELEM)
#converts the isotopes to the same format as the other dataframes

RADAR_df['A ELEM'] = [combine_RADAR(RADAR_A[i], str(RADAR_ELEM[i])) 
                      for i in range(RADAR_df.shape[0])]
#Note that this only includes the isotopes in both AM_table and nubase
RADAR_isotopes = list(RADAR_df['A ELEM'])
RADAR_energies = list(RADAR_df['Energy (keV)'])
average_beta_decay_energy = []
decay_types, beta_decay_fractions, beta_decay_list = {}, {}, []
intersection = list(set(AM_table['A Elt.']).intersection(set(nubase_df['A El'])))

def get_isotope_info(isotope, info = None, isotope_column = None,
    dataset = {}, isotope_list = None, list_to_search = []):                  
  '''
  isotope_list and list_to_search are optional arguments.
  If list_to_search is not provided, then info must be provided.
  If isotope_list is not provided, then dataset and isotope_column
  must be provided.
  '''
  if isotope_list is None:
    isotope_list = list(dataset[isotope_column])
  row = 0
  try:
    row = isotope_list.index(isotope)
  except:
    print("Isotope not found in dataset")
  if len(list_to_search) == 0:
    try:
      list_to_search = list(dataset[info])
    except:
      print("info to search for not entered")
      return
  return list_to_search[row]

nubase_isotopes = list(nubase_df['A El'])
nubase_br_list = list(nubase_df['BR'])
for n in intersection:
  decay_distr =  re.split(r' |;|=|<|>|~', get_isotope_info(n, 
      isotope_list = nubase_isotopes, list_to_search = nubase_br_list))
  decay_types[n] = decay_distr
  try:
    average_beta_decay_energy.append(get_isotope_info(n, isotope_list= RADAR_isotopes, 
            list_to_search=RADAR_energies))
  except:
    average_beta_decay_energy.append('unknown')
    print("No RADAR data for " + n)
  try:
    abundance = float(decay_distr[decay_distr.index('B-') + 1]) / 100
  except:
    abundance = 0
  beta_decay_fractions[n] = abundance  
  beta_decay_list.append(abundance)

Isotope not found in dataset
Isotope not found in dataset
Isotope not found in dataset
Isotope not found in dataset
Isotope not found in dataset
Isotope not found in dataset
Isotope not found in dataset
Isotope not found in dataset
Isotope not found in dataset
Isotope not found in dataset
Isotope not found in dataset
Isotope not found in dataset
Isotope not found in dataset
Isotope not found in dataset
Isotope not found in dataset
Isotope not found in dataset
Isotope not found in dataset
Isotope not found in dataset
Isotope not found in dataset
Isotope not found in dataset
Isotope not found in dataset
Isotope not found in dataset
Isotope not found in dataset
Isotope not found in dataset
Isotope not found in dataset
Isotope not found in dataset
Isotope not found in dataset
Isotope not found in dataset
Isotope not found in dataset
Isotope not found in dataset
Isotope not found in dataset
Isotope not found in dataset
Isotope not found in dataset
Isotope not found in dataset
Isotope not fo

In [5]:
AM_symbols = list(AM_table['A Elt.'])
nubase_symbols = list(nubase_df['A El'])
AM_rows = [AM_symbols.index(n) for n in
intersection]
nubase_rows = [nubase_symbols.index(n) for
n in intersection]
e_t_dict = {}
e_t_dict['Beta-decay fraction'] = beta_decay_list
e_t_dict['Average beta decay energy'] = average_beta_decay_energy
e_t_dict['Beta-decay energy (keV)'] = list(
AM_table['Beta-decay energy (keV)'][AM_rows])
e_t_dict['Half life (years)'] = list(nubase_df['T in s'][nubase_rows])
e_t_dict['Isotope'] = list(list(intersection))
no_energy = [row for row, energy in enumerate(e_t_dict['Average beta decay energy']) if (energy == 'unknown' or energy == '*')]
no_half_life = [row for row, half_life in enumerate(e_t_dict['Half life (years)']) 
               if type(half_life) is str]
not_measured_rows = no_energy + no_half_life
len(not_measured_rows)
e_t_dataframe = pd.DataFrame(e_t_dict)
filtered_e_t_df = e_t_dataframe.drop(not_measured_rows, inplace = False)
filtered_e_t_df.to_csv('NuclideData.csv')
filtered_e_t_df

,Beta-decay fraction,Average beta decay energy,Beta-decay energy (keV),Half life (years),Isotope
0,0.0,1533.00,-1864.01,1.05838e-06,80Rb
1,0.0,5.69,5480,1.5844e-07,185Yb
2,0.0,6166.00,-335.062,0.0560438,253Es
4,1.0,5.69,3951,1.90129e-07,195Re
5,0.0,5.67,-6989.4,3.58773e-05,162Yb
...,...,...,...,...,...
3553,1.0,5.69,11561.2,6.05242e-09,58V
3554,1.0,79.50,702.72,0.00106662,127Te
3555,0.0,543.30,-5069.14,717000,26Al
3556,0.0,477.00,-9447.82,0.000634269,90Mo


#Find power density of beta decay chains.

In [6]:
beta_e_t_df = filtered_e_t_df[filtered_e_t_df['Beta-decay fraction'] == 1]
beta_e_t_df.to_csv('BetaDecayData.csv')
element_symbols = ['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 
'F', 'Ne', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar', 'K', 
'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 
'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 
'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 
'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 
'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm',
 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 
 'Hg', 'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 
 'Th', 'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 
 'Fm', 'Md', 'No', 'Lr', 'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt',
  'Ds', 'Rg', 'Cn', 'Nh', 'Fl', 'Mc', 'Lv', 'Ts', 'Og', 'N/A']
#to find the daughter nuclide we only need to increment by 1 in the element symbols.
s = element_symbols
#remove numbers from string
beta_e_t_df['Daugher Nucleus'] = [re.sub('\D+', '', n) + s[s.index(re.sub('\d+', '', n))+1] 
if re.sub('\d+', '', n) in s else 'N/A' for n in beta_e_t_df['Isotope'] ]
nuclide_df = beta_e_t_df

half_lives = nuclide_df['Half life (years)'].to_numpy() 
nuclide_df["e Folding Time (seconds)"] = half_lives * units.year.to(units.s) / np.log(2)

def get_isotope_info(isotope, info = None, isotope_column = None,
    dataset = {}, isotope_list = None, lists_to_search = []):                  
  '''
  isotope_list and list_to_search are optional arguments.
  If list_to_search is not provided, then info must be provided.
  If isotope_list is not provided, then dataset and isotope_column
  must be provided.
  '''
  if isotope_list is None:
    isotope_list = list(dataset[isotope_column])
  row = isotope_list.index(isotope)
  if len(lists_to_search) == 0:
    try:
      lists_to_search = list(dataset[info])
    except:
      print("info to search for not entered")
      return
  try: #only works if there are multiple specified lists to search 
    return [target_list[row] for target_list in lists_to_search]
  except:
    return lists_to_search[row]



isotope_list = list(nuclide_df['Isotope'])
lambda_list = list(nuclide_df['e Folding Time (seconds)'])
decay_energy_list = list(nuclide_df['Average beta decay energy'])
daughter_list = list(nuclide_df['Daugher Nucleus'])

def make_decay_chain(isotope, isotope_list, lambda_list, decay_energy_list, daughter_list):
    '''
    This function takes the isotope, e-folding times, and daughter nuclides
    and returns the decay chain.
    '''
    decay_chain = {}
    decay_chain[isotope] = get_isotope_info(isotope, isotope_list = isotope_list, 
        lists_to_search = (lambda_list, decay_energy_list, daughter_list))
    while True:
        isotope = decay_chain[isotope][2]
        try:
            decay_chain[isotope] = get_isotope_info(isotope, isotope_list = isotope_list,
                lists_to_search = (lambda_list, decay_energy_list, daughter_list))
            if(isotope == decay_chain[isotope][2]):
                return decay_chain
        except:
            return pd.DataFrame(decay_chain, index= ("e-Folding Time (seconds)", 
                                "Average beta-decay energy", "Daughter")).transpose()

#Quickly calculating the decay rate of the i-th generation nuclide
def formulate_decay_rate(e_folding_times):
    '''
    The e-folding times must be in a numpy array.
    Returns a formula for the decay rate of each generation. 
    '''
    exponent_array = -1 / e_folding_times
    decay_rates = len(exponent_array)  * [sym.N(0)]
    decay_rates[0] = sym.exp(t * exponent_array[0]) * exponent_array[0]
    for index, L in enumerate(exponent_array[1:]):
        decay_rates[index+1] = decay_rates[index] * (1 + sym.exp(t * L) * L) 
    return decay_rates

def eval_decay_rates(decay_rates, time_array):
    '''
    This function takes the formula for the decay rate of each generation
    and substitutes each value in the time array for t.
    Each decay_rate must be a sympy expression. 
    https://docs.sympy.org/ 
    Rewrite so that the outer loop is over the time array and the inner loop
    is over the generations. This can be done by using (suggested by copilot: 
    the np.meshgrid function) or a 2d array (my first thought)
    '''
    try: #will only evaluate if decay_rates is an array of sympy expressions
        evaluated_decay_rates = [np.array([formula.subs(t, time) for formula in decay_rates])
                                    for time in time_array]
    except:
        evaluated_decay_rates = [decay_rates.subs(t, time) for time in time_array]
    return evaluated_decay_rates

#Calculating the power density of a decay chain
def calc_power_density(decay_rates, decay_energies, initial_mass):
    '''
    decay_rates must be a numpy array in moles/second
    decay_energies must be a numpy array in keV/decay
    This function takes the decay rates and energies of the decay chain.
    Returns the power density in watts/g as a function of time
    '''
    power_density =  [sum(decay_rate_t * decay_energies) for decay_rate_t in decay_rates]
    '''
    At every different value for time, 
    store the sum of the product of each generation's decay rate and decay energy.
    '''
    power_density = np.array(power_density)
    #convert to W/g
    power_density *=  units.keV.to(units.J) * float(constants.N_A * units.mol) / initial_mass
    return np.abs(power_density.astype(float))
    
def calculate_all_power_densities(all_decay_chains, time_array, mean = True):
    '''
    This function takes the isotope, e-folding times, and daughter nuclides
    and returns power density as a function of time for each decay chain.
    The optional boolean argument mean determines whether the whole time series is returned
    or only its mean. 
    '''
    if mean:
        all_power_densities = [np.mean(calc_power_density(
                                    decay_rates = eval_decay_rates(
                                                    formulate_decay_rate(chain['e-Folding Time (seconds)']), 
                                                    time_array), 
                                    decay_energies = np.array(chain['Average beta-decay energy']),
                                    initial_mass = float(re.sub('\D+', '', chain.index[0]))
                                )) for chain in all_decay_chains]
    else:
        all_power_densities = [calc_power_density(
                                    decay_rates = eval_decay_rates(
                                                    formulate_decay_rate(chain['e-Folding Time (seconds)']), 
                                                    time_array), 
                                    decay_energies = np.array(chain['Average beta-decay energy']),
                                    initial_mass = float(re.sub('\D+', '', chain.index[0]))
                                ) for chain in all_decay_chains]
    return all_power_densities

In [7]:
time_array = np.logspace(1, 9.5, 100)
beta_decay_chains = [make_decay_chain(isotope, isotope_list, lambda_list, decay_energy_list, daughter_list) 
                    for isotope in isotope_list]
beta_decay_chains

[      e-Folding Time (seconds) Average beta-decay energy Daughter
 195Re                  8.65617                      5.69    195Os
 195Os                  562.651                      5.69    195Ir
 195Ir                  11893.6                       296    195Pt,
       e-Folding Time (seconds) Average beta-decay energy Daughter
 238Pa                  197.361                      5.69     238U,
       e-Folding Time (seconds) Average beta-decay energy Daughter
 110Rh                  4.83303                      5.69    110Pd,
       e-Folding Time (seconds) Average beta-decay energy Daughter
 151Ba                  0.24093                      5.69    151La
 151La                 0.670853                      5.69    151Ce
 151Ce                  2.53914                      5.69    151Pr
 151Pr                  27.2669                      5.69    151Nd
 151Nd                  1076.83                      36.7    151Pm
 151Pm                   147501                        46  

In [23]:
beta_power_density_time_series = pd.DataFrame(calculate_all_power_densities(
                    beta_decay_chains, time_array, mean = False))
beta_power_densities_dict = {}
for row, chain in enumerate(beta_decay_chains):
    beta_power_densities_dict[chain.index[0]] = beta_power_density_time_series.iloc[row,:]
beta_power_densities_dict['Time (seconds)'] = time_array
beta_power_densities_df = pd.DataFrame.from_dict(
                            beta_power_densities_dict).set_index('Time (seconds)')
beta_power_densities_df.transpose()

In [49]:
def binary_search(arr, low, high, x):
 
    # Check base case
    if high >= low:
 
        mid = int((high + low) / 2)
 
        # If element is present at the middle itself
        if arr[mid] == x:
            return mid
 
        # If element is smaller than mid, then it can only
        # be present in left subarray
        elif arr[mid] > x:
            return binary_search(arr, low, mid - 1, x)
 
        # Else the element can only be present in right subarray
        else:
            return binary_search(arr, mid + 1, high, x)
 
    else:
        # Element is not present in the array
        return high, low

def sort_power_densities(power_densities_df, 
            time = -1, time_array  = [], min_index = [],
             want_max = True):
    '''
    The time is in years and the power densitites are in W/g.

    The time is the specified time at which we want to sort the power densities. 

    If the time is not specified, it will be assumed that power_densities is has only 1 column.

    power_densities_df is the dataframe of power densities. It must be specified. 

    time_array is the array of times. If not specified, it will be assumed that the time_array
    
    is the index of the power_densities_df.

    min_index is the index of the time_array at which we want to start sorting. If not specified,
    
    it will be found using the function named binary_search.

    want_max is a boolean that indicates if we want the smallest time larger than the requested value.
    '''
    if(type(min_index) is list):
        if(time == -1):
            return power_densities_df.sort_values(ascending = False)
        if(len(time_array) == 0):
            time_array = power_densities_df.index.values            
        if(want_max):
            min_index = max(binary_search(time_array, 0, len(time_array)-1, time))
        else:
            min_index = min(binary_search(time_array, 0, len(time_array)-1, time))
    return power_densities_df.iloc[min_index,:].sort_values(ascending = False)

maximum_beta_power_densities = {}
maximum_beta_power_densities['Power Density (W/g)'] = [sort_power_densities(beta_power_densities_df,
                         min_index = row)[0] for row in range(beta_power_densities_df.shape[0])]
maximum_beta_power_densities['Isotope'] = [sort_power_densities(beta_power_densities_df,
                         min_index = row).index[0] for row in range(beta_power_densities_df.shape[0])]
maximum_beta_power_densities['Time (seconds)'] = beta_power_densities_df.index
#maximum_power_densities['Decay Chain'] = [make_decay_chain(isotope, 
#    isotope_list, lambda_list, decay_energy_list, daughter_list) for isotope in maximum_power_densities['Isotope']]
maximum_beta_power_densities_df = pd.DataFrame(maximum_beta_power_densities)
maximum_beta_power_densities_df

,Power Density (W/g),Isotope,Time (seconds)
0,3.161781e+07,39S,1.000000e+01
1,2.771481e+07,39S,1.218593e+01
2,2.360392e+07,39S,1.484968e+01
3,2.014914e+07,28Al,1.809572e+01
4,1.974316e+07,28Al,2.205131e+01
...,...,...,...
95,2.635877e-01,32Si,1.434054e+09
96,2.522778e-01,32Si,1.747528e+09
97,2.391495e-01,32Si,2.129525e+09
98,2.240714e-01,32Si,2.595024e+09


In [50]:
def plot_maximum_power_densities(maximum_power_densities_df, time = 0, file_name = -1):
    if(time != 0):
        min_row = max(binary_search(maximum_power_densities_df['Time (seconds)'], 0,
                            maximum_power_densities_df.shape[0]-1, time))
        maximum_power_densities_df = maximum_power_densities_df.iloc[min_row:,:]
    fig = px.scatter(maximum_power_densities_df, x = 'Time (seconds)', y = 'Power Density (W/g)',
                    hover_name= 'Isotope', log_x = True, log_y = True, 
                    color = 'Isotope', color_continuous_scale = 'algae')
    fig.show()
    if type(file_name) is not int:
        fig.write_image(file_name)
    
plot_maximum_power_densities(maximum_beta_power_densities_df, .1, 
    file_name = subprocess.os.getcwd() + '\\maximum_beta_power_densities_0.1_years.PDF')
#export figure as PDF

In [9]:
make_decay_chain("28Al", isotope_list, lambda_list, 
                decay_energy_list, daughter_list)

,e-Folding Time (seconds),Average beta-decay energy,Daughter
28Al,194.331,1247.2,28Si
